# **R-Squared Disentangled**

### Import Libraries

In [4]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

pd.set_option('display.max_columns', 500)

### Import Data

In [5]:
dummy = pd.read_excel('../../Dados/result_PCA_indicador_dia.xlsx')
dummy

,Data,daily_index,Sentiment Score,PCA_Normalized
0,2023-06-30,56,0.131037,57.099674
1,2023-06-29,54,0.192296,54.899763
2,2023-06-28,62,0.095973,63.699619
3,2023-06-27,59,0.139035,60.399683
4,2023-06-26,55,0.195269,55.999767
...,...,...,...,...
1960,2018-02-05,11,0.090658,7.599657
1961,2018-02-04,24,0.017391,21.899541
1962,2018-02-03,40,0.350227,39.500001
1963,2018-02-02,15,0.092335,11.999656


## *R-Squared*

In [6]:
fg = dummy.drop(['daily_index', 'Sentiment Score'], axis=1).set_index('Data').sort_values(by='Data', ascending=True)
fg['weekly_mean'] = fg['PCA_Normalized'].rolling(7).mean()
fg['sentiment'] = 0

fg.loc[(fg['weekly_mean'] <= 24), 'sentiment'] = 'Extreme Fear'
fg.loc[(fg['weekly_mean'] >24) & (fg['weekly_mean'] <= 44), 'sentiment'] = 'Fear'
fg.loc[(fg['weekly_mean'] > 44) & (fg['weekly_mean'] <= 55), 'sentiment'] = 'Neutral'
fg.loc[(fg['weekly_mean'] > 55) & (fg['weekly_mean'] <= 75), 'sentiment'] = 'Greed'
fg.loc[(fg['weekly_mean'] > 75), 'sentiment'] = 'Extreme Greed'

# fg.loc[(fg['weekly_mean'] <= 44), 'sentiment'] = 'Fear'
# fg.loc[(fg['weekly_mean'] > 44) & (fg['weekly_mean'] <= 55), 'sentiment'] = 'Neutral'
# fg.loc[fg['weekly_mean'] > 55, 'sentiment'] = 'Greed'

# fg.loc[(fg['weekly_mean'] <= 50), 'sentiment'] = 'Fear'
# fg.loc[(fg['weekly_mean'] > 50), 'sentiment'] = 'Greed'

# fg.loc[(fg['weekly_mean'] > 24) & (fg['weekly_mean'] <= 75), 'sentiment'] = 'Neutral'
# fg.loc[(fg['weekly_mean'] <= 24) | (fg['weekly_mean'] > 75), 'sentiment'] = 'Extreme'

fg

,PCA_Normalized,weekly_mean,sentiment
Data,,,
2018-02-01,28.499527,NaN,0
2018-02-02,11.999656,NaN,0
2018-02-03,39.500001,NaN,0
2018-02-04,21.899541,NaN,0
2018-02-05,7.599657,NaN,0
...,...,...,...
2023-06-26,55.999767,61.342569,Greed
2023-06-27,60.399683,62.913969,Greed
2023-06-28,63.699619,63.385373,Greed


In [7]:
tech = pd.read_excel('../../Dados/tech_signals_w.xlsx')
tech['Date'] = pd.to_datetime(tech['Date'])
tech

,Date,Adj Close,Volume,MA_1_9,MA_1_12,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_9,MOM_12,VOL_1_9,VOL_1_12,VOL_2_9,VOL_2_12,VOL_3_9,VOL_3_12
0,2015-03-02,274.354004,242425100,1,1,1,1,1,0,1,0,1,1,1,1,1,1
1,2015-03-09,286.393005,259502000,1,1,1,1,1,1,1,0,1,1,1,1,1,1
2,2015-03-16,267.959991,206778200,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,2015-03-23,242.712997,179497700,0,0,1,1,1,1,1,0,1,1,1,1,1,1
4,2015-03-30,260.597992,150120700,1,1,0,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,2023-10-02,27935.089844,82183775698,1,1,1,1,1,0,0,0,0,0,0,0,0,0
449,2023-10-09,27159.652344,72673654487,1,0,1,1,1,1,1,0,0,0,0,0,0,0
450,2023-10-16,29993.896484,113402383094,1,1,1,1,1,1,1,1,0,0,0,0,0,0
451,2023-10-23,33086.234375,38363572311,1,1,1,1,1,1,1,1,1,0,1,0,0,0


In [8]:
df = tech.merge(fg, how='left', left_on='Date', right_on='Data')
df.dropna(inplace=True)
df.drop('PCA_Normalized', axis=1, inplace=True)
df

,Date,Adj Close,Volume,MA_1_9,MA_1_12,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_9,MOM_12,VOL_1_9,VOL_1_12,VOL_2_9,VOL_2_12,VOL_3_9,VOL_3_12,weekly_mean,sentiment
154,2018-02-12,10551.799805,53626569728,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33.999736,Fear
155,2018-02-19,9664.730469,55388419584,0,0,0,0,0,0,0,0,0,0,0,0,0,0,63.385416,Greed
156,2018-02-26,11512.599609,48902649856,1,0,0,0,0,0,0,0,0,0,0,0,0,0,44.528131,Neutral
157,2018-03-05,9578.629883,49671590400,0,0,1,0,0,0,0,0,0,0,0,0,0,0,46.570830,Neutral
158,2018-03-12,8223.679688,42075919360,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41.385200,Fear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,2023-05-29,27119.066406,91351657572,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50.499777,Neutral
431,2023-06-05,25940.167969,116498927148,0,0,0,0,0,0,0,0,0,0,0,0,0,0,52.385402,Neutral
432,2023-06-12,26336.212891,92905085602,0,0,0,0,0,0,0,0,0,0,0,0,0,0,48.928165,Neutral
433,2023-06-19,30480.261719,138005623534,1,1,1,1,0,0,1,1,1,1,0,0,0,0,46.099552,Neutral


In [9]:
rf = pd.read_excel('../../Dados/risk_free_d.xlsx')
rf['r_free_day'] = rf['r_free_day'].ffill()
rf['r_free_w'] = rf['r_free_day'].rolling(7).mean()
rf = rf[['observation_date', 'r_free_w']]
rf['observation_date'] = pd.to_datetime(rf['observation_date'])
rf.head(10)

,observation_date,r_free_w
0,2018-11-06,NaN
1,2018-11-07,NaN
2,2018-11-08,NaN
3,2018-11-09,NaN
4,2018-11-12,NaN
5,2018-11-13,NaN
6,2018-11-14,0.000087
7,2018-11-15,0.000087
8,2018-11-16,0.000087
9,2018-11-19,0.000087


In [10]:
dummies = pd.get_dummies(df['sentiment'])
df = pd.concat([df.drop('sentiment', axis=1), dummies], axis=1)
df['Any'] = [True]*len(df)
df['Returns'] = df['Adj Close'].pct_change()
df = df.merge(rf, how='left', left_on='Date', right_on='observation_date').dropna()
df.drop('observation_date', axis=1, inplace=True)
df['risk_premium'] = df['Returns'] - df['r_free_w']
df.head(10)

,Date,Adj Close,Volume,MA_1_9,MA_1_12,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_9,MOM_12,VOL_1_9,VOL_1_12,VOL_2_9,VOL_2_12,VOL_3_9,VOL_3_12,weekly_mean,Extreme Fear,Extreme Greed,Fear,Greed,Neutral,Any,Returns,r_free_w,risk_premium
39,2018-11-19,4009.969971,42533970000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,32.271121,False,False,True,False,False,True,-0.286931,0.000087,-0.287019
40,2018-11-26,4139.877930,42945276472,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11.685265,True,False,False,False,False,True,0.032396,0.000088,0.032308
41,2018-12-03,3614.234375,38386467708,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12.313996,True,False,False,False,False,True,-0.126971,0.000090,-0.127061
42,2018-12-10,3252.839111,29869247431,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11.056732,True,False,False,False,False,True,-0.099992,0.000092,-0.100084
43,2018-12-17,3998.980225,46021506218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10.270987,True,False,False,False,False,True,0.229381,0.000091,0.229290
44,2018-12-24,3865.952637,39249734918,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25.356855,False,False,True,False,False,True,-0.033265,0.000093,-0.033359
45,2018-12-31,4076.632568,34343716582,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24.099564,False,False,True,False,False,True,0.054496,0.000095,0.054401
46,2019-01-07,3552.953125,37523454177,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33.371148,False,False,True,False,False,True,-0.128459,0.000095,-0.128554
47,2019-01-14,3601.013672,38588596987,0,0,0,0,0,0,0,0,1,0,0,0,0,0,26.299654,False,False,True,False,False,True,0.013527,0.000095,0.013432
48,2019-01-21,3583.965820,36949379050,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27.556807,False,False,True,False,False,True,-0.004734,0.000094,-0.004829


In [11]:
r2 = {}

for i in range(3, 17):
    y = df['risk_premium'][1:df.shape[0]].values
    x = df.iloc[0:df.shape[0]-1, i].values
    x = sm.add_constant(x)  
    model = sm.OLS(y, x).fit()
    e = model.resid
    Ic = []
    for j in range(18, 24):
        r_squared = 1 - (np.sum(df.iloc[1:df.shape[0], j] * e**2) / np.sum(df.iloc[1:df.shape[0], j] * ((y - np.mean(y))**2)))
        print(f"Out-of-Sample R² for technical indicator {df.columns[i]} - {df.columns[j]}: {r_squared}")

        Ic.append(r_squared)
    r2[df.columns[i]] = Ic

Out-of-Sample R² for technical indicator MA_1_9 - Extreme Fear: 0.01527558467446788
Out-of-Sample R² for technical indicator MA_1_9 - Extreme Greed: 0.021713576955717517
Out-of-Sample R² for technical indicator MA_1_9 - Fear: 0.02959974777627039
Out-of-Sample R² for technical indicator MA_1_9 - Greed: 0.0023880331432962754
Out-of-Sample R² for technical indicator MA_1_9 - Neutral: 0.006230500879124912
Out-of-Sample R² for technical indicator MA_1_9 - Any: 0.018715961292381622
Out-of-Sample R² for technical indicator MA_1_12 - Extreme Fear: -0.030191566502423495
Out-of-Sample R² for technical indicator MA_1_12 - Extreme Greed: 0.018114223354213643
Out-of-Sample R² for technical indicator MA_1_12 - Fear: 0.026997556568491787
Out-of-Sample R² for technical indicator MA_1_12 - Greed: 0.005873355036004835
Out-of-Sample R² for technical indicator MA_1_12 - Neutral: 0.012831441473882488
Out-of-Sample R² for technical indicator MA_1_12 - Any: 0.010001407182678501
Out-of-Sample R² for technical

In [12]:
r2_df = pd.DataFrame(r2)
r2_df = r2_df.T 
r2_df.columns = ['Extreme Fear', 'Extreme Greed', 'Fear', 'Greed', 'Neutral', 'Any']
r2_df

,Extreme Fear,Extreme Greed,Fear,Greed,Neutral,Any
MA_1_9,0.015276,0.021714,0.029600,0.002388,0.006231,0.018716
MA_1_12,-0.030192,0.018114,0.026998,0.005873,0.012831,0.010001
MA_2_9,-0.002302,0.017426,0.010527,0.003352,0.010442,0.008623
MA_2_12,-0.017610,0.016797,0.019979,0.003569,0.004422,0.008304
MA_3_9,-0.007477,0.018174,0.001047,0.030745,0.024402,0.010102
MA_3_12,-0.015074,0.016068,0.001235,0.027326,0.017909,0.007098
MOM_9,-0.076177,0.039097,0.055844,0.017249,0.042114,0.021038
MOM_12,-0.021417,0.012366,0.013594,0.005593,-0.002622,0.003946
VOL_1_9,-0.006237,0.036653,0.011318,0.003610,0.056973,0.017633
VOL_1_12,-0.017100,0.017060,0.013019,0.000185,0.037167,0.009287


In [13]:
import plotly.express as px

In [14]:
r2_df.index = ['MA(1,9)','MA(1,12)','MA(2,9)','MA(2,12)','MA(3,9)','MA(3,12)','MOM(9)','MOM(12)','VOL(1,9)','VOL(1,12)','VOL(2,9)','VOL(2,12)','VOL(3,9)','VOL(3,12)']

cores2 = [px.colors.qualitative.G10[8], px.colors.qualitative.Set1[4], 
          px.colors.qualitative.Plotly[9], px.colors.qualitative.G10[7], px.colors.qualitative.Safe[3]]

fig = px.bar(r2_df*100, r2_df.index, ["Extreme Fear", "Fear", "Neutral", "Greed", "Extreme Greed"], 
             title="Figura 1. In-sample R² disentangled.", color_discrete_sequence=cores2,
             labels={"value": "R² (%)", "index": "", "variable": ""})
fig.add_hline(y=0.5, line_width=1.2, line_dash="dash", line_color="red")
fig.update_layout(barmode='group',bargroupgap=0, bargap=0.4, width=1200, height=400, title_x=0.05, title_y=0.03)
fig.update_xaxes(tickangle=25)
fig.show()